In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b3' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 300
start_epoch = 0
train_batch = 50
test_batch = 50
lr = 0.04
schedule = [75,150,225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b3/siamese_aug/100shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
# iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# constrastive
thresh = 0.5

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [9]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

# Dataset

In [10]:
train_dir = os.path.join(data_dir, '100_shot')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_ = SiameseNetworkDataset(datasets.ImageFolder(val_dir), transform=val_aug, should_invert=False)
val_loader = DataLoader(val_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [11]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    model.load_state_dict(pt)

In [12]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 10.89M


# Loss

In [13]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Train AUROC', 'Valid AUROC'])

In [15]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        
        loss = criterion(outputs0, outputs1, targets)
            
        # compute output
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
        
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))
        arc.update(auroc, inputs0.size(0))
        

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            loss = criterion(outputs0, outputs1, targets)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            # measure accuracy and record loss
            pred = outputs.data
            pred[pred < thresh] = 0.
            pred[pred >= thresh] = 1.
            prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
            auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
            
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))
            arc.update(auroc, inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
        
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(val_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_loader, model, criterion, epoch, use_cuda)

    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, train_auroc, test_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 300] LR: 0.040000
1/4 | Loss:0.9847 | top1:0.6200 | AUROC:0.5000
4/4 | Loss:0.7777 | top1:0.5600 | AUROC:0.4869
156/156 | Loss:0.3810 | top1:0.5069 | AUROC:0.5000

Epoch: [2 | 300] LR: 0.042400
1/4 | Loss:0.5787 | top1:0.5200 | AUROC:0.5200
4/4 | Loss:0.4804 | top1:0.5500 | AUROC:0.5401
156/156 | Loss:0.4616 | top1:0.5053 | AUROC:0.5000

Epoch: [3 | 300] LR: 0.044800
1/4 | Loss:0.4551 | top1:0.3600 | AUROC:0.3945
4/4 | Loss:0.3663 | top1:0.4550 | AUROC:0.4729
156/156 | Loss:0.4223 | top1:0.5023 | AUROC:0.5000

Epoch: [4 | 300] LR: 0.047200
1/4 | Loss:0.3808 | top1:0.4400 | AUROC:0.4439
4/4 | Loss:0.3273 | top1:0.5000 | AUROC:0.4981
156/156 | Loss:0.4744 | top1:0.4905 | AUROC:0.5000

Epoch: [5 | 300] LR: 0.049600
1/4 | Loss:0.3064 | top1:0.5600 | AUROC:0.5705
4/4 | Loss:0.3493 | top1:0.5300 | AUROC:0.5405
156/156 | Loss:0.4576 | top1:0.5090 | AUROC:0.5000

Epoch: [6 | 300] LR: 0.052000
1/4 | Loss:0.3341 | top1:0.5000 | AUROC:0.5000
4/4 | Loss:0.3028 | top1:0.5250 | AUROC:0.

4/4 | Loss:0.2898 | top1:0.5300 | AUROC:0.5104
156/156 | Loss:0.4502 | top1:0.5022 | AUROC:0.5000

Epoch: [48 | 300] LR: 0.152800
1/4 | Loss:0.2909 | top1:0.5600 | AUROC:0.5600
4/4 | Loss:0.3170 | top1:0.4600 | AUROC:0.4654
156/156 | Loss:0.4562 | top1:0.4947 | AUROC:0.5000

Epoch: [49 | 300] LR: 0.155200
1/4 | Loss:0.3550 | top1:0.3600 | AUROC:0.3892
4/4 | Loss:0.3259 | top1:0.4500 | AUROC:0.4668
156/156 | Loss:0.4703 | top1:0.4967 | AUROC:0.5000

Epoch: [50 | 300] LR: 0.157600
1/4 | Loss:0.3336 | top1:0.4600 | AUROC:0.4294
4/4 | Loss:0.3615 | top1:0.4550 | AUROC:0.4721
156/156 | Loss:0.4868 | top1:0.4926 | AUROC:0.5000

Epoch: [51 | 300] LR: 0.160000
1/4 | Loss:0.3090 | top1:0.5800 | AUROC:0.5499
4/4 | Loss:0.3099 | top1:0.5650 | AUROC:0.5426
156/156 | Loss:0.4749 | top1:0.5004 | AUROC:0.5000

Epoch: [52 | 300] LR: 0.160000
1/4 | Loss:0.3192 | top1:0.4800 | AUROC:0.4952
4/4 | Loss:0.3133 | top1:0.4800 | AUROC:0.4880
156/156 | Loss:0.4688 | top1:0.4950 | AUROC:0.5000

Epoch: [53 | 300

1/4 | Loss:0.2544 | top1:0.6000 | AUROC:0.5844
4/4 | Loss:0.2687 | top1:0.5450 | AUROC:0.5531
156/156 | Loss:0.4240 | top1:0.5015 | AUROC:0.5000

Epoch: [95 | 300] LR: 0.015203
1/4 | Loss:0.2738 | top1:0.5000 | AUROC:0.5230
4/4 | Loss:0.2920 | top1:0.4900 | AUROC:0.4989
156/156 | Loss:0.4205 | top1:0.5032 | AUROC:0.5000

Epoch: [96 | 300] LR: 0.015166
1/4 | Loss:0.2591 | top1:0.6000 | AUROC:0.6103
4/4 | Loss:0.2812 | top1:0.5150 | AUROC:0.5243
156/156 | Loss:0.4305 | top1:0.4927 | AUROC:0.5000

Epoch: [97 | 300] LR: 0.015128
1/4 | Loss:0.2867 | top1:0.4400 | AUROC:0.4759
4/4 | Loss:0.2740 | top1:0.5000 | AUROC:0.5197
156/156 | Loss:0.4178 | top1:0.5060 | AUROC:0.5000

Epoch: [98 | 300] LR: 0.015090
1/4 | Loss:0.2913 | top1:0.4600 | AUROC:0.4667
4/4 | Loss:0.2764 | top1:0.4950 | AUROC:0.4930
156/156 | Loss:0.4229 | top1:0.4978 | AUROC:0.5000

Epoch: [99 | 300] LR: 0.015050
1/4 | Loss:0.2865 | top1:0.4400 | AUROC:0.4416
4/4 | Loss:0.2690 | top1:0.5350 | AUROC:0.5367
156/156 | Loss:0.4161

4/4 | Loss:0.2954 | top1:0.4800 | AUROC:0.5073
156/156 | Loss:0.4213 | top1:0.4958 | AUROC:0.5000

Epoch: [141 | 300] LR: 0.012770
1/4 | Loss:0.2631 | top1:0.4600 | AUROC:0.4663
4/4 | Loss:0.2768 | top1:0.4800 | AUROC:0.4797
156/156 | Loss:0.4116 | top1:0.5065 | AUROC:0.5000

Epoch: [142 | 300] LR: 0.012702
1/4 | Loss:0.2759 | top1:0.5200 | AUROC:0.5362
4/4 | Loss:0.2885 | top1:0.5100 | AUROC:0.5028
156/156 | Loss:0.4175 | top1:0.4973 | AUROC:0.5000

Epoch: [143 | 300] LR: 0.012634
1/4 | Loss:0.3048 | top1:0.5000 | AUROC:0.5146
4/4 | Loss:0.2730 | top1:0.5150 | AUROC:0.5258
156/156 | Loss:0.4114 | top1:0.5044 | AUROC:0.5000

Epoch: [144 | 300] LR: 0.012566
1/4 | Loss:0.2925 | top1:0.4200 | AUROC:0.4475
4/4 | Loss:0.2751 | top1:0.5000 | AUROC:0.5145
156/156 | Loss:0.4165 | top1:0.4953 | AUROC:0.5000

Epoch: [145 | 300] LR: 0.012497
1/4 | Loss:0.2794 | top1:0.5600 | AUROC:0.5484
4/4 | Loss:0.2765 | top1:0.5000 | AUROC:0.4984
156/156 | Loss:0.4128 | top1:0.5010 | AUROC:0.5000

Epoch: [146

156/156 | Loss:0.4192 | top1:0.4947 | AUROC:0.5000

Epoch: [187 | 300] LR: 0.000925
1/4 | Loss:0.2977 | top1:0.4200 | AUROC:0.4135
4/4 | Loss:0.2829 | top1:0.4500 | AUROC:0.4484
156/156 | Loss:0.4186 | top1:0.4955 | AUROC:0.5000

Epoch: [188 | 300] LR: 0.000917
1/4 | Loss:0.2754 | top1:0.6000 | AUROC:0.6071
4/4 | Loss:0.2722 | top1:0.5500 | AUROC:0.5486
156/156 | Loss:0.4194 | top1:0.4949 | AUROC:0.5000

Epoch: [189 | 300] LR: 0.000909
1/4 | Loss:0.2796 | top1:0.4600 | AUROC:0.4599
4/4 | Loss:0.2632 | top1:0.5400 | AUROC:0.5518
156/156 | Loss:0.4212 | top1:0.4937 | AUROC:0.5000

Epoch: [190 | 300] LR: 0.000900
1/4 | Loss:0.3468 | top1:0.4200 | AUROC:0.4286
4/4 | Loss:0.2893 | top1:0.4850 | AUROC:0.4830
156/156 | Loss:0.4237 | top1:0.4886 | AUROC:0.5000

Epoch: [191 | 300] LR: 0.000892
1/4 | Loss:0.3235 | top1:0.5400 | AUROC:0.5667
4/4 | Loss:0.2790 | top1:0.5600 | AUROC:0.5496
156/156 | Loss:0.4170 | top1:0.4972 | AUROC:0.5000

Epoch: [192 | 300] LR: 0.000884
1/4 | Loss:0.2705 | top1:0

1/4 | Loss:0.2798 | top1:0.4400 | AUROC:0.4439
4/4 | Loss:0.2843 | top1:0.4900 | AUROC:0.4895
156/156 | Loss:0.4113 | top1:0.5058 | AUROC:0.5000

Epoch: [234 | 300] LR: 0.000054
1/4 | Loss:0.2402 | top1:0.5000 | AUROC:0.5113
4/4 | Loss:0.2846 | top1:0.4600 | AUROC:0.4689
156/156 | Loss:0.4088 | top1:0.5081 | AUROC:0.5000

Epoch: [235 | 300] LR: 0.000053
1/4 | Loss:0.2930 | top1:0.4400 | AUROC:0.4327
4/4 | Loss:0.2716 | top1:0.5150 | AUROC:0.5009
156/156 | Loss:0.4146 | top1:0.5021 | AUROC:0.5000

Epoch: [236 | 300] LR: 0.000052
1/4 | Loss:0.2848 | top1:0.5000 | AUROC:0.4757
4/4 | Loss:0.2735 | top1:0.5350 | AUROC:0.5104
156/156 | Loss:0.4166 | top1:0.4992 | AUROC:0.5000

Epoch: [237 | 300] LR: 0.000051
1/4 | Loss:0.2495 | top1:0.5600 | AUROC:0.5513
4/4 | Loss:0.2778 | top1:0.4550 | AUROC:0.4805
156/156 | Loss:0.4128 | top1:0.5058 | AUROC:0.5000

Epoch: [238 | 300] LR: 0.000051
1/4 | Loss:0.2517 | top1:0.5000 | AUROC:0.4901
4/4 | Loss:0.2833 | top1:0.4650 | AUROC:0.4563
156/156 | Loss:0

4/4 | Loss:0.2686 | top1:0.5250 | AUROC:0.5301
156/156 | Loss:0.4158 | top1:0.5012 | AUROC:0.5000

Epoch: [280 | 300] LR: 0.000022
1/4 | Loss:0.3151 | top1:0.3800 | AUROC:0.4075
4/4 | Loss:0.2942 | top1:0.4750 | AUROC:0.4842
156/156 | Loss:0.4162 | top1:0.5000 | AUROC:0.5000

Epoch: [281 | 300] LR: 0.000021
1/4 | Loss:0.2692 | top1:0.5600 | AUROC:0.5600
4/4 | Loss:0.2815 | top1:0.5250 | AUROC:0.5304
156/156 | Loss:0.4088 | top1:0.5082 | AUROC:0.5000

Epoch: [282 | 300] LR: 0.000021
1/4 | Loss:0.2830 | top1:0.5200 | AUROC:0.4833
4/4 | Loss:0.2740 | top1:0.5100 | AUROC:0.5025
156/156 | Loss:0.4108 | top1:0.5073 | AUROC:0.5000

Epoch: [283 | 300] LR: 0.000020
1/4 | Loss:0.2966 | top1:0.5400 | AUROC:0.5400
4/4 | Loss:0.2731 | top1:0.4850 | AUROC:0.4665
156/156 | Loss:0.4156 | top1:0.5022 | AUROC:0.5000

Epoch: [284 | 300] LR: 0.000019
1/4 | Loss:0.2825 | top1:0.5800 | AUROC:0.5017
4/4 | Loss:0.2601 | top1:0.5450 | AUROC:0.5283
156/156 | Loss:0.4155 | top1:0.5000 | AUROC:0.5000

Epoch: [285